# Data Exploration

## Default imports

In [103]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import sklearn
import requests

## Video Image Split

In [2]:
!ls ./example_dataset/images

VIRAT_S_000002_fr_0.2


In [3]:
def filter_file_extension(filename):
    
    file_extension = 'mp4'
    
    if file_extension in filename:
        return True
    else:
        return False

def get_videos_from_folder(path):
    
    all_items = os.listdir(path)
    only_videos = [video_name for video_name in filter(filter_file_extension, all_items)]
    video_filenames = [video_name.split('.')[0] for video_name in only_videos]
    videos_file_paths = [os.path.join(path, video_fp) for video_fp in only_videos]
    
    return videos_file_paths, video_filenames

In [4]:
video_file_paths, video_filenames = get_videos_from_folder('./example_dataset')

In [55]:
def get_frame(sec, video_object):
    '''
        Based on the given sec, extract the resulting image from the video_object.
    '''
    video_object.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
    success, image = video_object.read()
    return success, image
    

def write_images_from_video(video_path, video_filename, frame_rate):
    '''
        Splits the video by the frame_rate, shown in the video_path variable into different frames,
        then writes the resulting images in a folder with the  video's filename.
        Creates the folder if it exits.
    
    '''
    video_object = cv2.VideoCapture(video_path)
    sec = 0
    count = 0
    frame_rate = 1 / frame_rate # looking at the actual time from second to the next
    success = 1
    
    while success:
        
        success, image = get_frame(sec, video_object)
        
        # create the folder if not exists
        video_file_name = f'{video_filename}_fr_{frame_rate}'
        folder_path = f'./poc_dataset/images/{video_file_name}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # write the image
        cv2.imwrite(os.path.join(folder_path, f'{video_filename}_frame_{np.round(count,2)}.png'), image)
        
        sec = sec + frame_rate
        count += 1
    
    print(f'File {video_filename} processed.')

In [56]:
write_images_from_video(video_file_paths[0], video_filenames[0], 5)

File VIRAT_S_000002 processed.


## Utils
Various helper functions that are used in general.

In [7]:
def load_csv_datasets(path):
    files = os.listdir(path)
    
    data = {}
    for file in files:
        filename = file.split('.')[0]
        filepath = os.path.join(path, file)
        data[filename] = pd.read_csv(filepath)
    
    return data

## People Counting with Object Recognition Methods

[Reference](https://machinelearningmastery.com/object-recognition-with-deep-learning/?fbclid=IwAR3QHzwV3iteB2tE7EJo3GkNTT_v7loLqHtCqYuH5nopySIkzvTmHiUa-H0)

### 1. Faster R-CNN
* [Reference of Faster R-CNN](https://arxiv.org/pdf/1506.01497.pdf)  
* [Reference of implementation](https://towardsdatascience.com/faster-r-cnn-object-detection-implemented-by-keras-for-custom-data-from-googles-open-images-125f62b9141a)
* [Reference Jupyter Notebook for the implementation](https://github.com/RockyXu66/Faster_RCNN_for_Open_Images_Dataset_Keras/blob/master/frcnn_train_vgg.ipynb)

#### 1.1 Getting The Data:

#### 1.1.1 Downloading the dataset information with the corresponding bounding boxes

In [7]:
!wget https://datasets.figure-eight.com/figure_eight_datasets/open-images/train-annotations-bbox.csv && \
!wget https://datasets.figure-eight.com/figure_eight_datasets/open-images/train-images-boxable.csv

--2019-11-26 21:53:32--  https://datasets.figure-eight.com/figure_eight_datasets/open-images/train-annotations-bbox.csv
Resolving datasets.figure-eight.com (datasets.figure-eight.com)... 52.200.149.96, 3.227.119.162, 3.227.227.96
Connecting to datasets.figure-eight.com (datasets.figure-eight.com)|52.200.149.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1,1G) [text/csv]
Saving to: ‘train-annotations-bbox.csv’

train-annotations-b 100%[===================>]   1,11G   448KB/s    in 72m 55s 

2019-11-26 23:06:28 (267 KB/s) - ‘train-annotations-bbox.csv’ saved [1194033454/1194033454]



In [11]:
datasets = load_csv_datasets('./training_dataset/')

In [20]:
list(datasets.keys())

['train-images-boxable',
 'train-annotations-bbox',
 'class-descriptions-boxable']

#### 1.1.2 Downloading the actual dataset

In [73]:
cldesc_df = datasets['class-descriptions-boxable']
cldesc_df.columns = ['label_id', 'class']
# get the Label Name that I need - person
label_id = cldesc_df.loc[cldesc_df['class'] == 'Person', 'label_id'].values[0]

In [57]:
trannbb_df = datasets['train-annotations-bbox']
trimb_df = datasets['train-images-boxable']

In [66]:
# Creating an image id from the name
trimb_df['ImageID'] = trimb_df['image_name'].apply(lambda x: x.split('.')[0])
trimb_df.head()

,image_name,image_url,ImageID
0,e39871fd9fd74f55.jpg,https://requestor-proxy.figure-eight.com/figur...,e39871fd9fd74f55
1,f18b91585c4d3f3e.jpg,https://requestor-proxy.figure-eight.com/figur...,f18b91585c4d3f3e
2,ede6e66b2fb59aab.jpg,https://requestor-proxy.figure-eight.com/figur...,ede6e66b2fb59aab
3,ed600d57fcee4f94.jpg,https://requestor-proxy.figure-eight.com/figur...,ed600d57fcee4f94
4,ff47e649b23f446d.jpg,https://requestor-proxy.figure-eight.com/figur...,ff47e649b23f446d


In [89]:
num_images = 100000
person_bb_df = trannbb_df[trannbb_df['LabelName'] == label_id] # getting a filtered dataset of only images that contain persons
image_ids = list(trannbb_df.loc[trannbb_df['LabelName'] == label_id, 'ImageID'].unique())[:num_images] # get a list with the image ids

In [93]:
# filtered df based on the image ids
filtered_trimb_df = trimb_df[trimb_df['ImageID'].isin(image_ids)]

In [107]:
def pull_write_img(url, filename, path):
    r = requests.get(url)
    filepath = os.path.join(path, filename + '.jpg')
    if r.status_code == 200:
        with open(filepath, 'wb') as w:
            w.write(r.content)
            
def download_images(df, path):
    # hardcoding column names is not a good idea... 
    for imid, url in zip(df['ImageID'].values, df['image_url'].values):
        pull_write_img(url, imid, path)
    
    print('Done!')

In [ ]:
download_images(filtered_trimb_df, './training_dataset/images')